# Phase 3: Indicator Engine & Basic Strategies

Build the technical indicator library and implement the strategies used by most traders.

This notebook covers:
- **Moving Averages**: SMA, EMA, WMA
- **Oscillators**: RSI, MACD, Stochastic
- **Volatility**: Bollinger Bands, ATR
- **Volume**: OBV, VWAP
- **Trend**: ADX, Parabolic SAR
- **Basic Strategies**: Trend/range trading, scalping, breakout, mean reversion

---

```bash
pip install pandas numpy matplotlib
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

def generate_ohlcv(n=500, mu=0.08, sigma=0.20, start=100.0):
    dt = 1/252
    returns = np.random.normal(mu * dt, sigma * np.sqrt(dt), n)
    close = start * np.exp(np.cumsum(returns))
    noise = sigma * np.sqrt(dt) * 0.5
    dates = pd.date_range('2023-01-01', periods=n, freq='B')
    df = pd.DataFrame({
        'open': close * (1 + np.random.normal(0, noise, n)),
        'high': close * (1 + np.abs(np.random.normal(0, noise, n))),
        'low':  close * (1 - np.abs(np.random.normal(0, noise, n))),
        'close': close,
        'volume': np.random.lognormal(10, 1, n).astype(int)
    }, index=dates)
    return df

df = generate_ohlcv(500)
print(f"Generated {len(df)} candles, price ${df['close'].iloc[0]:.2f} → ${df['close'].iloc[-1]:.2f}")

---
## 3.1 Moving Averages

The foundation of most trading indicators. They smooth price data to reveal trends.

- **SMA** (Simple): Equal weight to all periods
- **EMA** (Exponential): More weight to recent prices
- **WMA** (Weighted): Linearly increasing weights

In [ ]:
def sma(series: pd.Series, period: int) -> pd.Series:
    return series.rolling(window=period).mean()

def ema(series: pd.Series, period: int) -> pd.Series:
    return series.ewm(span=period, adjust=False).mean()

def wma(series: pd.Series, period: int) -> pd.Series:
    weights = np.arange(1, period + 1)
    return series.rolling(window=period).apply(
        lambda x: np.dot(x, weights) / weights.sum(), raw=True)

# Calculate and plot
close = df['close']

fig, ax = plt.subplots(figsize=(14, 7))
ax.plot(close.index, close, color='gray', alpha=0.5, linewidth=0.8, label='Close')
ax.plot(close.index, sma(close, 20), label='SMA(20)', linewidth=1.5)
ax.plot(close.index, ema(close, 20), label='EMA(20)', linewidth=1.5)
ax.plot(close.index, wma(close, 20), label='WMA(20)', linewidth=1.5)
ax.plot(close.index, sma(close, 50), label='SMA(50)', linewidth=1.5, linestyle='--')
ax.set_title('Moving Average Comparison', fontsize=14)
ax.set_ylabel('Price ($)')
ax.legend()
plt.tight_layout()
plt.show()

print("Notice: EMA reacts faster to price changes than SMA. WMA is in between.")

### Exercise 3.1

1. Plot SMA(10) vs SMA(200). Where do they cross? These "golden cross" and "death cross" points are common signals.
2. How much lag does SMA(50) have vs EMA(50)? Measure the average number of candles of delay at turning points.
3. Implement a **Hull Moving Average**: `HMA = WMA(2*WMA(n/2) - WMA(n), sqrt(n))`. How does it compare?

In [ ]:
# YOUR CODE HERE


---
## 3.2 RSI (Relative Strength Index)

RSI measures the magnitude of recent price changes to evaluate overbought or oversold conditions.

- **RSI > 70**: Overbought (potential sell signal)
- **RSI < 30**: Oversold (potential buy signal)
- **RSI = 50**: Neutral

In [ ]:
def rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    avg_gain = gain.ewm(alpha=1/period, min_periods=period).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

rsi_14 = rsi(close, 14)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True, gridspec_kw={'height_ratios': [2, 1]})

ax1.plot(close.index, close, color='steelblue')
# Highlight overbought/oversold zones
overbought = close.where(rsi_14 > 70)
oversold = close.where(rsi_14 < 30)
ax1.scatter(close.index, overbought, color='red', s=10, alpha=0.5, label='RSI > 70')
ax1.scatter(close.index, oversold, color='green', s=10, alpha=0.5, label='RSI < 30')
ax1.set_ylabel('Price ($)')
ax1.set_title('RSI(14) Analysis', fontsize=14)
ax1.legend()

ax2.plot(rsi_14.index, rsi_14, color='purple', linewidth=1)
ax2.axhline(y=70, color='red', linestyle='--', alpha=0.5)
ax2.axhline(y=30, color='green', linestyle='--', alpha=0.5)
ax2.fill_between(rsi_14.index, 70, rsi_14.where(rsi_14 > 70), color='red', alpha=0.2)
ax2.fill_between(rsi_14.index, 30, rsi_14.where(rsi_14 < 30), color='green', alpha=0.2)
ax2.set_ylabel('RSI')
ax2.set_ylim(0, 100)

plt.tight_layout()
plt.show()

---
## 3.3 MACD (Moving Average Convergence Divergence)

MACD shows the relationship between two EMAs. Signals come from:
- **MACD line crossing signal line** (bullish when MACD crosses above)
- **MACD crossing zero** (trend direction)
- **Histogram divergence** (momentum changes)

In [ ]:
def macd(series: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9) -> Tuple[pd.Series, pd.Series, pd.Series]:
    ema_fast = ema(series, fast)
    ema_slow = ema(series, slow)
    macd_line = ema_fast - ema_slow
    signal_line = ema(macd_line, signal)
    histogram = macd_line - signal_line
    return macd_line, signal_line, histogram

macd_line, signal_line, histogram = macd(close)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True, gridspec_kw={'height_ratios': [2, 1]})

ax1.plot(close.index, close, color='steelblue', label='Close')
ax1.plot(close.index, ema(close, 12), color='orange', alpha=0.5, linewidth=0.8, label='EMA(12)')
ax1.plot(close.index, ema(close, 26), color='red', alpha=0.5, linewidth=0.8, label='EMA(26)')
ax1.set_ylabel('Price ($)')
ax1.set_title('MACD Analysis', fontsize=14)
ax1.legend()

ax2.plot(macd_line.index, macd_line, color='blue', linewidth=1.2, label='MACD')
ax2.plot(signal_line.index, signal_line, color='red', linewidth=1, label='Signal')
colors = ['green' if h >= 0 else 'red' for h in histogram]
ax2.bar(histogram.index, histogram, color=colors, alpha=0.4, width=1)
ax2.axhline(y=0, color='gray', linewidth=0.5)
ax2.set_ylabel('MACD')
ax2.legend()

plt.tight_layout()
plt.show()

---
## 3.4 Bollinger Bands & ATR

Volatility indicators that adapt to market conditions.

- **Bollinger Bands**: SMA ± k × standard deviation. Price touching bands can signal overbought/oversold.
- **ATR (Average True Range)**: Measures volatility in absolute terms. Used for stop placement and position sizing.

In [ ]:
def bollinger_bands(series: pd.Series, period: int = 20, num_std: float = 2.0):
    mid = sma(series, period)
    std = series.rolling(window=period).std()
    upper = mid + num_std * std
    lower = mid - num_std * std
    pct_b = (series - lower) / (upper - lower)  # %B: where price is within bands
    bandwidth = (upper - lower) / mid  # bandwidth: how wide bands are
    return upper, mid, lower, pct_b, bandwidth

def atr(df: pd.DataFrame, period: int = 14) -> pd.Series:
    high_low = df['high'] - df['low']
    high_close = abs(df['high'] - df['close'].shift())
    low_close = abs(df['low'] - df['close'].shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    return true_range.rolling(window=period).mean()


upper, mid, lower, pct_b, bandwidth = bollinger_bands(close)
atr_14 = atr(df)

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1, 1]})

axes[0].plot(close.index, close, color='steelblue', linewidth=1)
axes[0].plot(close.index, mid, color='orange', linewidth=0.8)
axes[0].fill_between(close.index, upper, lower, alpha=0.15, color='orange')
axes[0].plot(close.index, upper, color='orange', linewidth=0.5, linestyle='--')
axes[0].plot(close.index, lower, color='orange', linewidth=0.5, linestyle='--')
axes[0].set_ylabel('Price ($)')
axes[0].set_title('Bollinger Bands (20, 2σ) & ATR(14)', fontsize=14)

axes[1].plot(pct_b.index, pct_b, color='purple', linewidth=0.8)
axes[1].axhline(y=1.0, color='red', linestyle='--', alpha=0.5)
axes[1].axhline(y=0.0, color='green', linestyle='--', alpha=0.5)
axes[1].axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)
axes[1].set_ylabel('%B')
axes[1].set_ylim(-0.3, 1.3)

axes[2].plot(atr_14.index, atr_14, color='red', linewidth=1)
axes[2].set_ylabel('ATR ($)')
axes[2].set_xlabel('Date')

plt.tight_layout()
plt.show()

---
## 3.5 Basic Strategy: MA Crossover (Trend Following)

The simplest trend strategy: buy when fast MA crosses above slow MA, sell when it crosses below.

In [ ]:
def ma_crossover_strategy(
    df: pd.DataFrame,
    fast_period: int = 10,   # try changing!
    slow_period: int = 30,   # try changing!
    use_ema: bool = True
) -> pd.DataFrame:
    """Generate signals for MA crossover strategy."""
    result = df.copy()
    ma_func = ema if use_ema else sma
    result['fast_ma'] = ma_func(df['close'], fast_period)
    result['slow_ma'] = ma_func(df['close'], slow_period)

    # Signal: 1 = long, -1 = short, 0 = flat
    result['signal'] = 0
    result.loc[result['fast_ma'] > result['slow_ma'], 'signal'] = 1
    result.loc[result['fast_ma'] < result['slow_ma'], 'signal'] = -1

    # Detect crossovers (signal changes)
    result['crossover'] = result['signal'].diff()

    # Strategy returns (next-day returns, since we trade on signal day's close)
    result['market_return'] = df['close'].pct_change()
    result['strategy_return'] = result['signal'].shift(1) * result['market_return']

    return result.dropna()


strat = ma_crossover_strategy(df, fast_period=10, slow_period=30)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9), sharex=True, gridspec_kw={'height_ratios': [2, 1]})

ax1.plot(strat.index, strat['close'], color='gray', alpha=0.5, linewidth=0.8)
ax1.plot(strat.index, strat['fast_ma'], color='blue', linewidth=1, label='Fast EMA(10)')
ax1.plot(strat.index, strat['slow_ma'], color='red', linewidth=1, label='Slow EMA(30)')

# Mark buy/sell signals
buys = strat[strat['crossover'] == 2]
sells = strat[strat['crossover'] == -2]
ax1.scatter(buys.index, buys['close'], marker='^', color='green', s=100, zorder=5, label='Buy')
ax1.scatter(sells.index, sells['close'], marker='v', color='red', s=100, zorder=5, label='Sell')
ax1.set_ylabel('Price ($)')
ax1.set_title('MA Crossover Strategy', fontsize=14)
ax1.legend()

# Cumulative returns comparison
cum_market = (1 + strat['market_return']).cumprod()
cum_strategy = (1 + strat['strategy_return']).cumprod()
ax2.plot(strat.index, cum_market, color='gray', label='Buy & Hold')
ax2.plot(strat.index, cum_strategy, color='steelblue', label='MA Crossover')
ax2.axhline(y=1.0, color='gray', linestyle=':', alpha=0.5)
ax2.set_ylabel('Cumulative Return')
ax2.set_xlabel('Date')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Buy & Hold return: {(cum_market.iloc[-1] - 1):.2%}")
print(f"Strategy return: {(cum_strategy.iloc[-1] - 1):.2%}")
print(f"Number of trades: {len(buys) + len(sells)}")

---
## 3.6 Mean Reversion Strategy (Bollinger Bands)

Buy when price touches the lower band (oversold), sell when it reaches the upper band (overbought). Works best in range-bound markets.

In [ ]:
def mean_reversion_strategy(
    df: pd.DataFrame,
    bb_period: int = 20,
    bb_std: float = 2.0,
    rsi_period: int = 14,
    rsi_oversold: float = 30,    # try changing!
    rsi_overbought: float = 70   # try changing!
) -> pd.DataFrame:
    result = df.copy()
    upper, mid, lower, pct_b, bw = bollinger_bands(df['close'], bb_period, bb_std)
    result['bb_upper'] = upper
    result['bb_mid'] = mid
    result['bb_lower'] = lower
    result['pct_b'] = pct_b
    result['rsi'] = rsi(df['close'], rsi_period)

    result['signal'] = 0
    # Buy: price below lower band AND RSI oversold
    result.loc[(result['close'] < result['bb_lower']) & (result['rsi'] < rsi_oversold), 'signal'] = 1
    # Sell: price above upper band AND RSI overbought
    result.loc[(result['close'] > result['bb_upper']) & (result['rsi'] > rsi_overbought), 'signal'] = -1

    # Forward-fill signals (stay in position until opposite signal)
    result['position'] = result['signal'].replace(0, np.nan).ffill().fillna(0)

    result['market_return'] = df['close'].pct_change()
    result['strategy_return'] = result['position'].shift(1) * result['market_return']

    return result.dropna()


mr = mean_reversion_strategy(df)

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [3, 1, 1]})

axes[0].plot(mr.index, mr['close'], color='steelblue', linewidth=1)
axes[0].fill_between(mr.index, mr['bb_upper'], mr['bb_lower'], alpha=0.1, color='orange')
buys = mr[mr['signal'] == 1]
sells = mr[mr['signal'] == -1]
axes[0].scatter(buys.index, buys['close'], marker='^', color='green', s=80, zorder=5)
axes[0].scatter(sells.index, sells['close'], marker='v', color='red', s=80, zorder=5)
axes[0].set_ylabel('Price ($)')
axes[0].set_title('Mean Reversion: Bollinger Bands + RSI', fontsize=14)

axes[1].plot(mr.index, mr['rsi'], color='purple', linewidth=0.8)
axes[1].axhline(y=70, color='red', linestyle='--', alpha=0.5)
axes[1].axhline(y=30, color='green', linestyle='--', alpha=0.5)
axes[1].set_ylabel('RSI')

cum_market = (1 + mr['market_return']).cumprod()
cum_strat = (1 + mr['strategy_return']).cumprod()
axes[2].plot(mr.index, cum_market, color='gray', label='Buy & Hold')
axes[2].plot(mr.index, cum_strat, color='steelblue', label='Mean Reversion')
axes[2].set_ylabel('Cumulative Return')
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"Strategy return: {(cum_strat.iloc[-1] - 1):.2%}")
print(f"Buy & Hold return: {(cum_market.iloc[-1] - 1):.2%}")
print(f"Buy signals: {len(buys)}, Sell signals: {len(sells)}")

---
## 3.7 Breakout Strategy

Enter when price breaks out of a consolidation range (Donchian channel breakout).

In [ ]:
def breakout_strategy(
    df: pd.DataFrame,
    lookback: int = 20,       # try 10, 20, 55
    exit_lookback: int = 10   # shorter lookback for exits
) -> pd.DataFrame:
    result = df.copy()
    result['upper_channel'] = df['high'].rolling(lookback).max()
    result['lower_channel'] = df['low'].rolling(lookback).min()
    result['exit_upper'] = df['high'].rolling(exit_lookback).max()
    result['exit_lower'] = df['low'].rolling(exit_lookback).min()

    position = 0
    signals = []
    for i in range(len(result)):
        if i < lookback:
            signals.append(0)
            continue
        prev_upper = result['upper_channel'].iloc[i-1]
        prev_lower = result['lower_channel'].iloc[i-1]
        current = result['close'].iloc[i]

        if position == 0:
            if current > prev_upper:
                position = 1  # breakout long
            elif current < prev_lower:
                position = -1  # breakout short
        elif position == 1:
            if current < result['exit_lower'].iloc[i-1]:
                position = 0  # exit long
        elif position == -1:
            if current > result['exit_upper'].iloc[i-1]:
                position = 0  # exit short
        signals.append(position)

    result['position'] = signals
    result['market_return'] = df['close'].pct_change()
    result['strategy_return'] = pd.Series(signals).shift(1).values * result['market_return']

    return result.dropna()


bo = breakout_strategy(df, lookback=20)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True, gridspec_kw={'height_ratios': [2, 1]})

ax1.plot(bo.index, bo['close'], color='steelblue', linewidth=1)
ax1.plot(bo.index, bo['upper_channel'], color='green', linewidth=0.8, linestyle='--', alpha=0.5)
ax1.plot(bo.index, bo['lower_channel'], color='red', linewidth=0.8, linestyle='--', alpha=0.5)

# Color background by position
for i in range(1, len(bo)):
    if bo['position'].iloc[i] == 1:
        ax1.axvspan(bo.index[i-1], bo.index[i], alpha=0.05, color='green')
    elif bo['position'].iloc[i] == -1:
        ax1.axvspan(bo.index[i-1], bo.index[i], alpha=0.05, color='red')
ax1.set_ylabel('Price ($)')
ax1.set_title('Donchian Channel Breakout (20/10)', fontsize=14)

cum_market = (1 + bo['market_return']).cumprod()
cum_strat = (1 + bo['strategy_return']).cumprod()
ax2.plot(bo.index, cum_market, color='gray', label='Buy & Hold')
ax2.plot(bo.index, cum_strat, color='steelblue', label='Breakout')
ax2.set_ylabel('Cumulative Return')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Strategy return: {(cum_strat.iloc[-1] - 1):.2%}")
print(f"Buy & Hold return: {(cum_market.iloc[-1] - 1):.2%}")

---
## 3.8 Strategy Comparison

Compare all three strategies on the same data.

In [ ]:
# Run all strategies
ma_strat = ma_crossover_strategy(df, 10, 30)
mr_strat = mean_reversion_strategy(df)
bo_strat = breakout_strategy(df, 20)

fig, ax = plt.subplots(figsize=(14, 7))

# Align to common start
start = max(ma_strat.index[0], mr_strat.index[0], bo_strat.index[0])

for strat, name, color in [
    (ma_strat, 'MA Crossover', 'blue'),
    (mr_strat, 'Mean Reversion', 'green'),
    (bo_strat, 'Breakout', 'orange'),
]:
    s = strat[strat.index >= start]
    cum = (1 + s['strategy_return']).cumprod()
    ax.plot(s.index, cum, label=name, color=color, linewidth=1.5)

# Buy & hold
bh = df[df.index >= start]['close']
ax.plot(bh.index, bh / bh.iloc[0], color='gray', linestyle='--', label='Buy & Hold')

ax.axhline(y=1.0, color='gray', linewidth=0.5)
ax.set_title('Strategy Comparison', fontsize=14)
ax.set_ylabel('Cumulative Return')
ax.set_xlabel('Date')
ax.legend()
plt.tight_layout()
plt.show()

### Exercise 3.8

1. Which strategy performs best in trending markets? Which in range-bound markets? Generate data with different characteristics and test.
2. Add a MACD-based strategy and include it in the comparison.
3. What happens when you apply 5x leverage to each strategy's returns? Recalculate and compare.
4. Load your own data and run all three strategies. Which works best for your asset?

In [ ]:
# YOUR CODE HERE
# my_data = pd.read_csv('my_data.csv', parse_dates=['date'], index_col='date')


---
## 3.9 Comprehension Check

1. Why does EMA react faster than SMA to price changes?
2. RSI reads 25. What does this tell you? What would you do as a mean reversion trader vs a trend follower?
3. The Bollinger Bands bandwidth is narrowing ("squeeze"). What typically follows?
4. Your MA crossover strategy has many whipsaws in a sideways market. What filter could you add to reduce false signals?
5. A breakout strategy enters 100 trades: 30 winners averaging +8%, 70 losers averaging -2%. Is this profitable? Calculate the expectancy.

In [ ]:
# YOUR ANSWERS HERE
